In [12]:
# Imports
import numpy as np
rng_seed = 368
np.random.seed(rng_seed)
from mongoengine import connect, Document, StringField, IntField, MapField, ListField
from tqdm import tqdm
import pymongo

In [13]:
# Connect to db
cursor = connect('testdb')

In [14]:
cursor

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

In [15]:
# Define schema for db
class Article(Document):
    cord_uid = StringField(required=True, primary_key=True)
    title = StringField(field=str, required=False)
    word_frequencies = MapField(field=IntField(), required=True)

class Word(Document):
    word = StringField(required=True, primary_key=True)
#     word = MapField(field=StringField(), required=True, primary_key=True)
    cord_uids = ListField(field=StringField(), required=True)

In [16]:
# Delete all articles
Article.objects().delete()
Word.objects().delete()

7

In [17]:
# Saving some articles to db
for i in tqdm(range(5), unit='article'):
    cord_uid = f'lalala_{i + 1}'
    article = Article(cord_uid=cord_uid)
    for j in range(5):
        word_str = f'word_{j + 1}'
        article.word_frequencies[word_str] = int(np.random.random() * 1000)
        article.title = 'dick'
    article.save()

for i in tqdm(range(5), unit='word'):
    word_str = f'word_{i + 1}'
    word = Word(word=word_str)
    for j in range(3):
        cord_uid = word_str+f' lalala_{j}'
        word.cord_uids.append(cord_uid)
    word.save()

100%|██████████| 5/5 [00:00<00:00, 988.15word/s]


In [18]:
client = pymongo.MongoClient('localhost', 27017, maxPoolSize=100000)
db = client['testdb']
coll = db.word
coll.insert_one({'_id':'fefe','cord_uids':['a','b','c']})
coll.find_one('fefe')

{'_id': 'fefe', 'cord_uids': ['a', 'b', 'c']}

In [22]:
Word.objects(word='fefe')[0].cord_uids

['a', 'b', 'c']

In [8]:
temp = Word(word='lala')
temp.cord_uids.extend(['1','2','3'])
temp.save()

temp2 = Word.objects(word='lala')[0]
temp.cord_uids

['1', '2', '3']

In [9]:
# Looking up specific cord_uid
print(Article.objects(cord_uid='lalala_1')[0].word_frequencies)
print(Article.objects(cord_uid='lalala_1')[0].title)

{'word_1': 415, 'word_2': 0, 'word_3': 790, 'word_4': 898, 'word_5': 185}
dick


In [10]:
# Looking up specific word
word_3_cord_uids = Word.objects(word='word_1')[0].cord_uids
word_3_cord_uids

['word_1 lalala_0', 'word_1 lalala_1', 'word_1 lalala_2']

In [11]:
for cord_uid in word_3_cord_uids[:5]:
    article = Article.objects(cord_uid=cord_uid)[0]
    print(article.word_frequencies)

IndexError: no such item for Cursor instance